In [1]:
!pip install torch-geometric

In [2]:
import torch

import pandas as pd

from torch_geometric.data import Data
from torch_geometric.nn   import GATConv
from tqdm                 import tqdm

In [3]:
# edges
df_G = pd.read_csv('network.csv', header = None)
df_G.columns = ['source', 'target', 'weight']
df_G.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 834421 entries, 0 to 834420
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   source  834421 non-null  int64  
 1   target  834421 non-null  int64  
 2   weight  834421 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 19.1 MB


In [4]:
# features
df_nodes = pd.read_csv('node_features.csv')
df_nodes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 677640 entries, 0 to 677639
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   user_id              677640 non-null  int64  
 1   user_rt              677640 non-null  int64  
 2   num_post             677640 non-null  int64  
 3   user_time_rt         677640 non-null  float64
 4   num_post_unverified  677640 non-null  int64  
 5   num_post_non-rumor   677640 non-null  int64  
 6   num_post_true        677640 non-null  int64  
 7   num_post_false       677640 non-null  int64  
 8   num_rt_unverified    677640 non-null  int64  
 9   num_rt_non-rumor     677640 non-null  int64  
 10  num_rt_true          677640 non-null  int64  
 11  num_rt_false         677640 non-null  int64  
 12  score                677640 non-null  float64
dtypes: float64(2), int64(11)
memory usage: 67.2 MB


In [5]:
df_nodes = df_nodes.drop(columns=['score'])

In [6]:
# they need to increment from 0
nodes = df_nodes['user_id'].tolist()
node_map = {node: idx for idx, node in enumerate(nodes)}

In [7]:
# edges tensor
edge_index = torch.tensor([[node_map[src] for src in df_G["source"]],
                           [node_map[tgt] for tgt in df_G["target"]]], dtype=torch.long)
edge_index.shape

torch.Size([2, 834421])

In [8]:
# weights tensor
edge_weight = torch.tensor(df_G["weight"].values, dtype=torch.float)
edge_weight.shape

torch.Size([834421])

In [9]:
# features tensor
node_features = torch.tensor(df_nodes.values, dtype=torch.float)
node_features.shape

torch.Size([677640, 12])

In [10]:
data = Data(x = node_features, edge_index=edge_index, edge_attr=edge_weight)
data

Data(x=[677640, 12], edge_index=[2, 834421], edge_attr=[834421])

In [11]:
# architecture
class GAT(torch.nn.Module):
    def __init__(self, in_channels, out_channels, heads=1):
        super(GAT, self).__init__()
        self.gat = GATConv(in_channels, out_channels, heads=heads, concat=False)


    def forward(self, x, edge_index, edge_attr):
        x, attention_weights = self.gat(x, edge_index, return_attention_weights=True)
        return x, attention_weights

In [12]:
model  = GAT(in_channels=node_features.shape[1], out_channels=2)
model

GAT(
  (gat): GATConv(12, 2, heads=1)
)

In [13]:
def train_model(model, lr = 0.01, epochs = 100):

  optimizer = torch.optim.Adam(model.parameters(), lr=lr)

  for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    node_embeddings, attention_weights = model(data.x, data.edge_index, data.edge_attr)
    loss = torch.mean(attention_weights[1])
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

  return attention_weights

In [14]:
attention_weights = train_model(model, epochs = 10)

Epoch 1, Loss: 0.4484439194202423


In [15]:
attention_weights[1].shape

torch.Size([1511092, 1])

In [16]:
attention_scores = attention_weights[1]

node_importance = torch.zeros(len(nodes))
for i, (src, dst) in tqdm(enumerate(data.edge_index.t()), desc = 'Aggregating Attention'):
    node_importance[src] += attention_scores[i].abs().sum()
    node_importance[dst] += attention_scores[i].abs().sum()

_, important_node_indices = torch.topk(node_importance, k=10)

print(f'Most Important Nodes: {important_node_indices}')

Aggregating Attention: 749847it [01:44, 7172.68it/s]


KeyboardInterrupt: 